In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image

In [3]:
from tensorflow.keras.models import load_model

cnn_a_loaded = load_model("saved_models/cnn_a.keras")
cnn_b_loaded = load_model("saved_models/cnn_b.keras")
cnn_c_loaded = load_model("saved_models/cnn_c.keras")

In [4]:
def load_images_for_test(folder_label_pairs, image_size=(224,224), pad_color=(255,255,255)):
    X, y, filenames = [], [], []
    for folder, label in folder_label_pairs:
        for fname in os.listdir(folder):
            if not fname.lower().endswith(('.jpg','.jpeg','.png')):
                continue
            try:
                img = Image.open(os.path.join(folder, fname)).convert("RGB")
                img.thumbnail(image_size, Image.Resampling.LANCZOS)
                background = Image.new("RGB", image_size, pad_color)
                offset = ((image_size[0]-img.width)//2, (image_size[1]-img.height)//2)
                background.paste(img, offset)
                arr = np.array(background)/255.0
                X.append(arr)
                y.append(label)
                filenames.append(fname)
            except Exception as e:
                print(f"⚠️ {fname} skipped: {e}")
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int32), filenames

In [5]:
test_folders = [
(r"E:\POC_Jain_Irrigation\data\A\test_cropped", 1),
(r"E:\POC_Jain_Irrigation\data\BC\test_cropped", 2),
(r"E:\POC_Jain_Irrigation\data\D\test_cropped", 3)
]
X_test_new, y_test_new, fnames = load_images_for_test(test_folders)
print(X_test_new.shape, y_test_new.shape)

(173, 224, 224, 3) (173,)


In [7]:
from sklearn.metrics import accuracy_score, classification_report

class_names = ["A", "BC", "D"]

# ============================================================
# ⚙️ Evaluate CNN Models on Real Test Images
# ============================================================
all_results = []  # to collect rows for CSV

for name, model in {
    "CNN_A": cnn_a_loaded,
    "CNN_B": cnn_b_loaded,
    "CNN_C": cnn_c_loaded
}.items():
    print(f"\n🔹 Testing {name} on real test images...")

    # --- Predictions ---
    probs = model.predict(X_test_new)  # shape: [n_samples, n_classes]
    y_pred = np.argmax(probs, axis=1)

    # If labels are 1-based (1, 2, 3), convert to 0-based
    y_true = y_test_new - 1  

    # --- Accuracy & Report ---
    acc = accuracy_score(y_true, y_pred)
    print(f"{name} Test Accuracy: {acc:.3f}")
    print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

    # ============================================================
    # 🔍 Per-sample Predictions with All Class Probabilities
    # ============================================================
    print(f"\n--- {name} Predictions with Probabilities ---")
    for i in range(len(X_test_new)):
        true_label = class_names[y_true[i]]
        pred_label = class_names[y_pred[i]]
        prob_str = " | ".join([f"{cls}: {probs[i][j]:.4f}" for j, cls in enumerate(class_names)])

        print(f"Sample {i+1:<4} | True: {true_label:<5} | Pred: {pred_label:<5}")
        print(f"  → Probabilities: {prob_str}")
        print("-" * 80)

        # Collect for CSV
        row = {
            "Model": name,
            "Sample_ID": i + 1,
            "True_Class": true_label,
            "Predicted_Class": pred_label,
            "Top_Probability": float(np.max(probs[i])),
        }
        for j, cls in enumerate(class_names):
            row[f"Prob_{cls}"] = float(probs[i][j])
        all_results.append(row)

# ============================================================
# 💾 Save All Results to CSV
# ============================================================
df = pd.DataFrame(all_results)
df.to_csv("cnn_predictions_with_probs.csv", index=False)
print("\n✅ Saved detailed predictions for all CNNs to 'cnn_predictions_with_probs.csv'")


🔹 Testing CNN_A on real test images...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
CNN_A Test Accuracy: 0.584
              precision    recall  f1-score   support

           A       0.42      0.80      0.55        20
          BC       0.98      0.48      0.64       130
           D       0.32      1.00      0.48        23

    accuracy                           0.58       173
   macro avg       0.57      0.76      0.56       173
weighted avg       0.83      0.58      0.61       173


--- CNN_A Predictions with Probabilities ---
Sample 1    | True: A     | Pred: A    
  → Probabilities: A: 1.0000 | BC: 0.0000 | D: 0.0000
--------------------------------------------------------------------------------
Sample 2    | True: A     | Pred: A    
  → Probabilities: A: 1.0000 | BC: 0.0000 | D: 0.0000
--------------------------------------------------------------------------------
Sample 3    | True: A     | Pred: A    
  → Probabilities: A: 1.0000 | BC: 0.0000 | D: 0.0000
------------------------